In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize config with toml file
c = sfp.SolutionFilesConfig('config_archive/china/CHN.toml')

from solution_file_processing.plots import create_plot_1a

[13:42:41 00:00] I:117:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240430_134241-CHN.log.
[13:42:42 00:00] I:149:__init__ - Initialized SolutionFilesConfig for config_archive/china/CHN.toml.


In [2]:
# would be good to make palettes etc based on the regions for consistency purposes
reg_ids = list(set(c.v.load_by_reg.reset_index()[c.GEO_COLS[0]].values))
reg_palette = {reg_ids[i]: IEA_PALETTE_16[i] for i in range(len(reg_ids))}

# Model palette
model_ids = list(set(c.v.load_by_reg.reset_index()['model'].values))
model_ids = [m for m in PRETTY_MODEL_NAMES if m in model_ids] + [m for m in model_ids if m not in PRETTY_MODEL_NAMES]
# Use extended palette so it can have more than 16 variables
model_palette = {model_ids[i]: IEA_PALETTE_16[i] for i in range(len(model_ids))}

# Regions and technologies will always be consistent this way. May need to be copied to other parts of the code
combined_palette = dict(STACK_PALETTE, **reg_palette, **model_palette)

[13:42:42 00:00] I:47:_drive_cache_wrapper - Loading from objects cache: node_yr_df.parquet (pd.DataFrame).


In [3]:
c.v.net_load_orig_ts

[13:42:52 00:10] I:44:_drive_cache_wrapper - Loading from objects cache: node_df.parquet (dd.DataFrame).
[13:42:55 00:02] I:47:_drive_cache_wrapper - Loading from objects cache: gen_yr_df.parquet (pd.DataFrame).
[13:42:55 00:00] I:44:_drive_cache_wrapper - Loading from objects cache: purch_df.parquet (dd.DataFrame).
[13:43:51 00:55] I:44:_drive_cache_wrapper - Loading from objects cache: reg_df.parquet (dd.DataFrame).
[13:45:45 01:54] I:44:_drive_cache_wrapper - Loading from objects cache: gen_df.parquet (dd.DataFrame).


value
model                    timestamp                         
01_China_2022_nMUT       2035-01-01 00:00:00  715021.850421
                         2035-01-01 01:00:00  772259.749907
                         2035-01-01 02:00:00  795741.826189
                         2035-01-01 03:00:00  812718.786844
                         2035-01-01 04:00:00  804504.000265
...                                                     ...
03_China_STEPS_2030_nMUT 2035-12-31 19:00:00  811271.724785
                         2035-12-31 20:00:00  791212.701537
                         2035-12-31 21:00:00  777547.389272
                         2035-12-31 22:00:00  865115.432866
                         2035-12-31 23:00:00  958826.230275

[26280 rows x 1 columns]